<a href="https://colab.research.google.com/github/fabiansep/TesisSMA/blob/main/graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from google.cloud import storage
from google.colab import userdata
from google.colab import drive
from google.colab import auth
from google.cloud import storage
import io


In [2]:
auth.authenticate_user()

In [4]:
# Configura el cliente de Google Cloud Storage
client = storage.Client()

In [5]:
# Define el nombre del bucket
bucket_name = 'tesis_data'
bucket = client.get_bucket(bucket_name)

In [8]:
# Función para leer CSV desde una carpeta específica del bucket
def read_csv_from_gcs(bucket, folder):
    blobs = bucket.list_blobs(prefix=folder)
    df_list = []
    for blob in blobs:
        if blob.name.endswith('.csv'):
            # Descarga el archivo CSV a un DataFrame
            content = blob.download_as_text()
            df = pd.read_csv(io.StringIO(content))  # Cambiado a io.StringIO
            df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

In [11]:
# Leer los archivos desde ambas carpetas
df_optical_depth = read_csv_from_gcs(bucket, 'comunas')

In [12]:
df_temperatura = read_csv_from_gcs(bucket, 'temperatura')

In [13]:
# Unir los DataFrames utilizando las columnas comunes
df_merged = pd.merge(
    df_optical_depth,
    df_temperatura,
    on=['Comuna', 'Region', 'fecha_desde', 'fecha_hasta'],
    suffixes=('_optical_depth', '_temperatura')
)


In [14]:

# Mantener solo las columnas necesarias
df_final = df_merged[['Comuna', 'Region', 'fecha_desde', 'fecha_hasta', 'valor_medio_optical_depth_047', 'valor_medio_temperatura']]

# Mostrar el resultado final
print(df_final.head())

  Comuna                            Region fecha_desde fecha_hasta  \
0  Alhué  Región Metropolitana de Santiago  2000-01-01  2000-01-07   
1  Alhué  Región Metropolitana de Santiago  2000-01-08  2000-01-14   
2  Alhué  Región Metropolitana de Santiago  2000-01-15  2000-01-21   
3  Alhué  Región Metropolitana de Santiago  2000-01-22  2000-01-28   
4  Alhué  Región Metropolitana de Santiago  2000-01-29  2000-02-04   

   valor_medio_optical_depth_047  valor_medio_temperatura  
0                            NaN                      NaN  
1                            NaN                      NaN  
2                            NaN                      NaN  
3                            NaN                      NaN  
4                            NaN                      NaN  


In [15]:
df_final.to_csv('df_final.csv', index=False)

In [16]:
def save_df_to_gcs(df, bucket, file_path):
    # Convertir el DataFrame a CSV en memoria
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Crear un blob en el bucket
    blob = bucket.blob(file_path)

    # Subir el contenido CSV al blob
    blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')

# Definir la ruta del archivo en la raíz del bucket
file_path = 'df_final.csv'  # Solo el nombre del archivo, ya que lo guardarás en la raíz

# Guardar el DataFrame en el bucket
save_df_to_gcs(df_final, bucket, file_path)
